In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import date

from sklearn.cluster import KMeans

In [2]:
path = Path.home() / 'OneDrive - Seagroup/kaggle_dataset/OnlineRetail.csv'
df = pd.read_csv(path, encoding='ISO-8859-1')

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['diff_date'] = (max(df['InvoiceDate']) - df['InvoiceDate']).dt.days
df['date_id'] = pd.to_datetime(df['InvoiceDate']).dt.strftime('%Y-%m-%d')
df.eval('amount = UnitPrice * Quantity', inplace=True)

print(df.shape)
df.head()

(541909, 11)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,diff_date,date_id,amount
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,373,2010-12-01,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,373,2010-12-01,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,373,2010-12-01,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,373,2010-12-01,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,373,2010-12-01,20.34


In [3]:
tx_6m = df[(df['date_id'] < '2011-09-01') & (df['date_id'] >= '2011-03-01')].reset_index(drop=True)
tx_next = df[(df['date_id'] >= '2011-09-01') & (df['date_id'] < '2011-12-01')].reset_index(drop=True)

In [4]:
tx_next_first_purchase = tx_next.groupby('CustomerID').InvoiceDate.min().rename('MinPurchaseDate').reset_index()
tx_last_purchase = tx_6m.groupby('CustomerID').InvoiceDate.max().rename('MaxPurchaseDate').reset_index()

tx_purchase_dates = pd.merge(tx_last_purchase, tx_next_first_purchase, on='CustomerID', how='left')
tx_purchase_dates['NextPurchaseDay'] = (tx_purchase_dates['MinPurchaseDate'] - tx_purchase_dates['MaxPurchaseDate']).dt.days
tx_purchase_dates.head()

,CustomerID,MaxPurchaseDate,MinPurchaseDate,NextPurchaseDay
0,12347.0,2011-08-02 08:48:00,2011-10-31 12:25:00,90.0
1,12348.0,2011-04-05 10:47:00,2011-09-25 13:13:00,173.0
2,12352.0,2011-03-22 16:08:00,2011-09-20 14:34:00,181.0
3,12353.0,2011-05-19 17:47:00,NaT,NaN
4,12354.0,2011-04-21 13:11:00,NaT,NaN


In [5]:
agg = {'diff_date': 'min', 'InvoiceNo': 'count','amount': 'sum'}
rfm = df.query(f"CustomerID == {tx_purchase_dates['CustomerID'].unique().tolist()}").groupby('CustomerID').agg(agg).reset_index()
rfm.columns = ['CustomerID', 'r', 'f', 'm']
rfm.head()

,CustomerID,r,f,m
0,12347.0,1,182,4310.00
1,12348.0,74,31,1797.24
2,12352.0,35,95,1545.41
3,12353.0,203,4,89.00
4,12354.0,231,58,1079.40


In [6]:
rfm.f.describe()

count    2845.000000
mean      120.590510
std       281.453937
min         1.000000
25%        24.000000
50%        59.000000
75%       134.000000
max      7983.000000
Name: f, dtype: float64

In [11]:
def clusterByKMean(X, k):
    kmeans = KMeans(n_clusters=k).fit(X)
    return kmeans.labels_

for i in ['r', 'f', 'm']:
    tx_purchase_dates[f'{i}_cluster'] = clusterByKMean(rfm[[i]], 4)
tx_purchase_dates = tx_purchase_dates.merge(rfm, on='CustomerID', how='left')

In [12]:
tx_purchase_dates

,CustomerID,MaxPurchaseDate,MinPurchaseDate,NextPurchaseDay,r_cluster,f_cluster,m_cluster,r,f,m
0,12347.0,2011-08-02 08:48:00,2011-10-31 12:25:00,90.0,1,2,0,1,182,4310.00
1,12348.0,2011-04-05 10:47:00,2011-09-25 13:13:00,173.0,3,2,0,74,31,1797.24
2,12352.0,2011-03-22 16:08:00,2011-09-20 14:34:00,181.0,1,2,0,35,95,1545.41
3,12353.0,2011-05-19 17:47:00,NaT,NaN,2,2,0,203,4,89.00
4,12354.0,2011-04-21 13:11:00,NaT,NaN,2,2,0,231,58,1079.40
...,...,...,...,...,...,...,...,...,...,...
2840,18280.0,2011-03-07 09:52:00,NaT,NaN,2,2,0,277,10,180.60
2841,18281.0,2011-06-12 10:53:00,NaT,NaN,0,2,0,180,7,80.82
2842,18282.0,2011-08-09 15:10:00,NaT,NaN,1,2,0,7,13,176.60
2843,18283.0,2011-07-14 13:20:00,2011-09-05 12:35:00,52.0,1,0,0,3,756,2094.88


In [16]:
tx_purchase_dates['OverallScore'] = tx_purchase_dates['r_cluster'] + tx_purchase_dates['f_cluster'] + tx_purchase_dates['m_cluster']
tx_purchase_dates.groupby('OverallScore')[['r','f','m']].mean()

,r,f,m
OverallScore,,,
0,147.000000,294.375000,2149.108750
1,12.290323,352.788856,5160.015161
2,158.604124,37.767010,1154.356474
3,19.654797,98.846686,2452.627320
4,231.196305,111.930716,1689.396215
5,77.552773,76.277281,2568.711665
6,41.166667,1428.666667,39492.413333
7,5.500000,1248.000000,100754.760000


In [22]:
#create a dataframe with CustomerID and Invoice Date
tx_day_order = tx_6m[['CustomerID','InvoiceDate']].copy()

#Convert Invoice Datetime to day
tx_day_order['InvoiceDay'] = tx_6m['InvoiceDate'].dt.date
tx_day_order = tx_day_order.sort_values(['CustomerID','InvoiceDate'])

#Drop duplicates
tx_day_order = tx_day_order.drop_duplicates(subset=['CustomerID','InvoiceDay'],keep='first')

#shifting last 3 purchase dates
tx_day_order['PrevInvoiceDate'] = tx_day_order.groupby('CustomerID')['InvoiceDay'].shift(1)
tx_day_order['T2InvoiceDate'] = tx_day_order.groupby('CustomerID')['InvoiceDay'].shift(2)
tx_day_order['T3InvoiceDate'] = tx_day_order.groupby('CustomerID')['InvoiceDay'].shift(3)
tx_day_order

,CustomerID,InvoiceDate,InvoiceDay,PrevInvoiceDate,T2InvoiceDate,T3InvoiceDate
42950,12347.0,2011-04-07 10:43:00,2011-04-07,NaN,NaN,NaN
115237,12347.0,2011-06-09 13:01:00,2011-06-09,2011-04-07,NaN,NaN
181286,12347.0,2011-08-02 08:48:00,2011-08-02,2011-06-09,2011-04-07,NaN
40678,12348.0,2011-04-05 10:47:00,2011-04-05,NaN,NaN,NaN
974,12352.0,2011-03-01 14:57:00,2011-03-01,NaN,NaN,NaN
...,...,...,...,...,...,...
205511,NaN,2011-08-24 09:00:00,2011-08-24,NaN,NaN,NaN
208022,NaN,2011-08-25 13:57:00,2011-08-25,NaN,NaN,NaN
208932,NaN,2011-08-26 11:47:00,2011-08-26,NaN,NaN,NaN
210956,NaN,2011-08-30 10:29:00,2011-08-30,NaN,NaN,NaN


In [20]:
tx_day_order.groupby('CustomerID')['InvoiceDay'].shift(1)

42950            NaN
115237    2011-04-07
181286    2011-06-09
40678            NaN
974              NaN
             ...    
205511           NaN
208022           NaN
208932           NaN
210956           NaN
214186           NaN
Name: InvoiceDay, Length: 8609, dtype: object